In [123]:
import pandas as pd
import pickle
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import os
import pickle
import numpy as np

In [124]:
df = pd.read_csv("/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/xgboost/evaluation_proccessed/proc_8_600_600.csv")

/tmp/ipykernel_685200/1049740314.py:1: DtypeWarning: Columns (129,131,132,134) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/xgboost/evaluation_proccessed/proc_8_600_600.csv")


In [125]:
df =df .drop(["Unnamed: 0"],axis=1)

In [126]:
df2 = df[df["accident_label"]]

df2.head()

,step,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,...,rolling_travel_time_6_22,rolling_travel_time_21_7,rolling_travel_time_1_16,rolling_travel_time_15_2,rolling_travel_time_19_23,rolling_travel_time_28_20,rolling_travel_time_32_16,rolling_travel_time_15_29,rolling_travel_time_36_18,rolling_travel_time_17_37
8141,8141,7.831831,2.565000,0.046163,4.638036,2.773333,0.130126,11.118799,6.115000,0.100365,...,75.313333,69.090580,235.0,NaN,130.0625,138.9,NaN,NaN,69.779412,53.5
8142,8142,7.831827,2.558333,0.046045,4.629533,2.775000,0.130245,11.134406,6.113333,0.100338,...,75.313333,69.359259,235.0,NaN,130.0625,138.9,NaN,NaN,69.779412,53.5
8143,8143,7.831826,2.551667,0.045927,4.620596,2.778333,0.130366,11.146702,6.113333,0.100338,...,75.108974,69.571970,235.0,NaN,130.0625,138.9,NaN,NaN,69.779412,53.5
8144,8144,7.826644,2.543333,0.045779,4.613208,2.780000,0.130501,11.157918,6.115000,0.100353,...,75.104938,70.038760,235.0,NaN,130.0625,138.9,NaN,NaN,69.779412,53.5
8145,8145,7.821533,2.535000,0.045632,4.607816,2.781667,0.130582,11.166930,6.118333,0.100407,...,74.529762,70.634921,235.0,NaN,130.0625,138.9,NaN,NaN,69.779412,52.0


In [127]:
df.shape

(842774, 159)

In [128]:
def standardize(col):
   return (col - col.mean()) / col.std()


In [129]:
incident_category = ["stalled_vehicle","multi_vehicle_collision"]
df2["label"] = None
for category in incident_category:
# Find rows where incident_edge is in the current set of edges
    mask = df2['incident_type'] == category
# Set the appropriate label column to 1 for these rows
    if category ==  "stalled_vehicle":
        df2.loc[mask, 'label'] = 0
    else:
        df2.loc[mask, 'label'] = 1

Y = df2["label"]
X = df2.drop(["step","incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label","label"],axis=1)


X = X.apply(standardize)


/tmp/ipykernel_685200/2941676736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["label"] = None


In [130]:
model_path = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/xgboost/saved_models/V24/incident_classification_xg_model.pkl"
xgb_model_loaded = pickle.load(open(model_path, "rb"))

In [131]:
X= X.values
Y= Y.values
y_pred = xgb_model_loaded.predict(X)
y_pred = list(y_pred)
y_test = list(Y)

accuracy = accuracy_score(y_test, y_pred)*100
precision = precision_score(y_test, y_pred,average='macro')
f1 = f1_score(y_test, y_pred,average='macro')
recall = recall_score(y_test, y_pred,average='macro')
print("Accuracy: %.2f%%" % (accuracy))
print(f"FAR {100-accuracy}")
print("precision: %.2f%%" % (precision * 100.0))
print("f1: %.2f%%" % (f1 * 100.0))
print("recall: %.2f%%" % (recall * 100.0))

Accuracy: 82.03%
FAR 17.965389461662127
precision: 75.49%
f1: 73.28%
recall: 71.81%
